In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import boto3

# Initialize Flask app and enable CORS
app = Flask(__name__)
CORS(app)

# Initialize Dask client
cluster = LocalCluster(n_workers=4)
client = Client(cluster)

# Set up AWS S3 connection
s3 = boto3.client('s3')
bucket_name = '608project'

def load_data(year):
    file_path = f's3://{bucket_name}/data_{year}.parquet'
    return dd.read_parquet(file_path)

def filter_data(data, boroughs, offenses, ethnicities, genders, age_categories):
    return data[
        (data['borough'].isin(boroughs)) &
        (data['offense'].isin(offenses)) &
        (data['ethnicity'].isin(ethnicities)) &
        (data['gender'].isin(genders)) &
        (data['age_category'].isin(age_categories))
    ]

@app.route('/filter', methods=['POST'])
def filter_data_endpoint():
    req = request.json
    years = req.get('years', [])
    boroughs = req.get('boroughs', [])
    offenses = req.get('offenses', [])
    ethnicities = req.get('ethnicities', [])
    genders = req.get('genders', [])
    age_categories = req.get('age_categories', [])

    # Load and concatenate data for the specified years
    data_frames = [load_data(year) for year in years]
    data = dd.concat(data_frames)

    # Filter the data
    filtered_data = filter_data(data, boroughs, offenses, ethnicities, genders, age_categories).compute()

    return jsonify(filtered_data.to_dict(orient='records'))

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
